<img width="150" alt="Logo_ER10" src="https://user-images.githubusercontent.com/3244249/151994514-b584b984-a148-4ade-80ee-0f88b0aefa45.png">

## Keras to ONNX conversion
This notebook shows how to convert your trained Keras model to ONNX, the generic format supported by DIANNA. <br>

The conversion is complete with the tf2onnx Python package, which supports both the SavedModel format and the older HDF5 (.h5 or .keras) format. It can convert multi-backend keras as well as tf.keras models.

In [1]:
import os

import numpy as np
import tensorflow as tf
from tensorflow import keras

import onnx
import onnxruntime as ort
# In addition to these imports, this notebook
# depends on tf2onnx. It is used from the command line.

Download and initialize built-in model.

In [2]:
model = keras.applications.mobilenet.MobileNet(weights='imagenet')

Evaluate model on some random input.

In [3]:
input_shape = [1] + model.inputs[0].shape[1:]  # input shape without a 1 for batch size, instead of None
input_data = np.random.normal(size=input_shape).astype(np.float32)
pred = model.predict(input_data)

Save keras model to SavedModel format.

In [4]:
savedmodel_dir = 'mysavedmodel'
tf.saved_model.save(model, savedmodel_dir)


FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.
INFO:tensorflow:Assets written to: mysavedmodel\assets


Convert to ONNX.

In [5]:
onnx_savedmodel = 'mysavedmodel.onnx'
!python -m tf2onnx.convert --saved-model {savedmodel_dir} --output {onnx_savedmodel} --signature_def serving_default --tag serve

2022-03-10 16:21:35.777938: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-03-10 16:21:35.777995: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
C:\Users\ChristiaanMeijer\anaconda3\envs\temp-dianna\lib\runpy.py:127: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2022-03-10 16:21:39.793252: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'nvcuda.dll'; dlerror: nvcuda.dll not found
2022-03-10 16:21:39.793284: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-10 16:21:39.798447: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retriev

Evaluate ONNX models and compare to keras model output.

In [6]:
# verify the ONNX model is valid
onnx_model = onnx.load(onnx_savedmodel)
onnx.checker.check_model(onnx_model)

# get ONNX predictions
sess = ort.InferenceSession(onnx_savedmodel)
input_name = sess.get_inputs()[0].name
output_name = sess.get_outputs()[0].name

onnx_input = {input_name: input_data}
pred_onnx = sess.run([output_name], onnx_input)[0]

print(np.allclose(pred_onnx, pred, atol=1e-5))

True
